# Network Medicine

***Lucas Goiriz Beltrán***&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;*Instituto de Biología Integrativa y de Sistemas (I2SysBio, UV - CSIC) & Departamento de Matemática Aplicada (UPV)*

***Alberto Conejero Casares***&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;*Departamento de Matemática Aplicada (UPV)*

---------------------------------------------------------------------------------------------------------------------

## Ejemplo: creación de un grafo a partir de estudios GWAS sobre el cáncer de mama

Hasta ahora, nos hemos dedicado a descargar grafos ya construídos.

La realidad sin embargo, es muy distinta. Veamos ahora un ejemplo donde acabaremos construyendo un grafo a partir de unos datos y unas relaciones.

La pregunta a responder es: en los estudios de GWAS, ¿cuáles son los procesos de los genes candidatos y cómo se relacionan estos procesos?

Los datos se encuentran en [este enlace](https://www.ebi.ac.uk/gwas/efotraits/EFO_0000305). Para descargarlos, vamos a emplear la `API` de GWAS Catalog.

Las librerías a emplear son las siguientes:

In [2]:
import json # Para leer respuestas de la api
import requests # Para hacer peticiones a la api
import numpy as np # Para construir matrices
import pandas as pd # Para construir tablas (tipo excel)
import networkx as nx # Para construir grafos
import netwulf as nw # Para visualizar el grafp
import xml.etree.ElementTree as ET # Para leer respuestas de la api

Ya sabéis que si no tenéis alguna de estas librerías, las podéis descargar mediante `conda` y/o `pip`.

Hagamos primero la consulta de los estudios de GWAS sobre el cáncer de mama. No os preocupéis si tarda un rato, ya que estas peticiones a servidores requieren tiempo. Tened paciencia.

In [3]:
# Dirección de la api junto al término que de cáncer de mama: EFO_0000305
url = r"https://www.ebi.ac.uk/gwas/rest/api/efoTraits/EFO_0000305/associations"

# Realizamos la consulta
consulta = json.loads(requests.get(url).text)

# Creamos data frame
df = pd.DataFrame(consulta["_embedded"]["associations"])

# Borramos las variables que no vamos a usar más
del consulta, url

Los datos descargados tienen el siguiente aspecto:

In [4]:
df

,riskFrequency,pvalueDescription,pvalueMantissa,pvalueExponent,multiSnpHaplotype,snpInteraction,snpType,standardError,range,description,orPerCopyNum,betaNum,betaUnit,betaDirection,loci,lastMappingDate,lastUpdateDate,pvalue,_links
0,0.34,None,4,-7,True,False,novel,NaN,None,None,NaN,NaN,None,None,"[{'haplotypeSnpCount': 3, 'description': '3-SN...",2021-06-24T22:55:40.000+0000,2021-06-24T22:55:40.000+0000,4.000000e-07,{'self': {'href': 'https://www.ebi.ac.uk/gwas/...
1,NR,(Total Mortality),6,-6,False,False,novel,NaN,[1.16-1.44],None,1.29,NaN,None,None,"[{'haplotypeSnpCount': None, 'description': 'S...",2021-06-24T17:40:45.000+0000,2021-06-24T17:40:46.000+0000,6.000000e-06,{'self': {'href': 'https://www.ebi.ac.uk/gwas/...
2,NR,(Total Mortality),1,-7,False,False,novel,NaN,[1.28-1.72],None,1.49,NaN,None,None,"[{'haplotypeSnpCount': None, 'description': 'S...",2021-06-24T19:34:09.000+0000,2021-06-24T19:34:10.000+0000,1.000000e-07,{'self': {'href': 'https://www.ebi.ac.uk/gwas/...
3,NR,(Recurrence),3,-7,False,False,novel,NaN,[1.25-1.64],None,1.43,NaN,None,None,"[{'haplotypeSnpCount': None, 'description': 'S...",2021-06-24T17:40:45.000+0000,2021-06-24T17:40:46.000+0000,3.000000e-07,{'self': {'href': 'https://www.ebi.ac.uk/gwas/...
4,0.40,,1,-10,False,False,novel,NaN,[1.07-1.42],None,1.20,NaN,None,None,"[{'haplotypeSnpCount': None, 'description': 'S...",2021-06-24T17:32:31.000+0000,2021-06-24T17:32:32.000+0000,1.000000e-10,{'self': {'href': 'https://www.ebi.ac.uk/gwas/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,NR,None,5,-16,False,False,novel,0.0108,[0.067-0.109],None,NaN,0.0880,unit,decrease,"[{'haplotypeSnpCount': None, 'description': 'S...",2021-12-10T01:50:53.000+0000,2021-12-10T11:56:00.000+0000,5.000000e-16,{'self': {'href': 'https://www.ebi.ac.uk/gwas/...
1822,NR,None,2,-8,False,False,novel,0.0489,[0.18-0.37],None,NaN,0.2728,unit,increase,"[{'haplotypeSnpCount': None, 'description': 'S...",2021-12-10T01:50:53.000+0000,2021-12-10T11:55:59.000+0000,2.000000e-08,{'self': {'href': 'https://www.ebi.ac.uk/gwas/...
1823,NR,None,3,-8,False,False,novel,0.0216,[0.077-0.162],None,NaN,0.1193,unit,increase,"[{'haplotypeSnpCount': None, 'description': 'S...",2021-12-10T01:50:53.000+0000,2021-12-10T11:55:58.000+0000,3.000000e-08,{'self': {'href': 'https://www.ebi.ac.uk/gwas/...
1824,NR,None,2,-10,False,False,novel,0.0296,[0.13-0.25],None,NaN,0.1893,unit,increase,"[{'haplotypeSnpCount': None, 'description': 'S...",2021-12-10T01:50:53.000+0000,2021-12-10T11:55:58.000+0000,2.000000e-10,{'self': {'href': 'https://www.ebi.ac.uk/gwas/...


Nuestro conjunto de datos tiene 1826 entradas. 
Lo que a nosotros realmente nos importa, se encuentra en la columna `loci`, donde entre otras cosas, se nos proporciona los identificadores de los genes en cuestión.

In [6]:
df_genes = df["loci"].apply( # Aplicamos función al dataframe ...
    lambda val: val[0]["authorReportedGenes"] # Tomamos los nombres de los genes...
        if val[0]["authorReportedGenes"] != [] # ... solamente en caso de que la lista sea no vacía...
        else np.nan # ... en caso contrario, ponemos un NaN
).to_frame() # Convierte la salida a data frame
df_genes

,loci
0,"[{'geneName': 'GLG1', 'entrezGeneIds': [{'entr..."
1,"[{'geneName': 'intergenic', 'entrezGeneIds': [..."
2,"[{'geneName': 'RAD51L1', 'entrezGeneIds': [], ..."
3,"[{'geneName': 'RAD51L1', 'entrezGeneIds': [], ..."
4,"[{'geneName': 'FGFR2', 'entrezGeneIds': [{'ent..."
...,...
1821,NaN
1822,NaN
1823,NaN
1824,NaN


Para empezar podemos ver que existen filas donde no tenemos información que identifique la región genómica. Además, vemos que dentro de `loci` teníamos información importante como `geneName`, `entrezGeneIds` y más... Esta información debería estar en columnas independientes.

In [7]:
df_genes = (
    df_genes
    .dropna() # Eliminamos los NaN
    .apply(
        lambda row: row['loci'][0].values(), # Extrae la información de cada diccionario
        axis=1, # Itera por filas
        result_type="expand" # Crea nuevas columnas con el resultado
    )
)

# Damos nuevos nombres a las columnas
df_genes.columns = ("geneName", "entrezGeneId", "ensemblGeneId")
df_genes

,geneName,entrezGeneId,ensemblGeneId
0,GLG1,[{'entrezGeneId': '2734'}],[{'ensemblGeneId': 'ENSG00000090863'}]
1,intergenic,[],[]
2,RAD51L1,[],[]
3,RAD51L1,[],[]
4,FGFR2,[{'entrezGeneId': '2263'}],[{'ensemblGeneId': 'ENSG00000066468'}]
...,...,...,...
1615,L3MBTL3,[{'entrezGeneId': '84456'}],[{'ensemblGeneId': 'ENSG00000198945'}]
1616,ADCY9,[{'entrezGeneId': '115'}],[{'ensemblGeneId': 'ENSG00000162104'}]
1617,BAIAP2,[{'entrezGeneId': '10458'}],[{'ensemblGeneId': 'ENSG00000175866'}]
1619,COX11,[{'entrezGeneId': '1353'}],[{'ensemblGeneId': 'ENSG00000166260'}]


Con el paso anterior, hemos reducido el conjunto de datos a 1428 entradas. Ya empieza a tener mejor aspecto... Podemos identificar varias cosas:
- Hay valores en `geneName` que refieren a zonas intergénicas, por lo tanto esto deberíamos descartarlo
- Hay que "extraer" los identificadores para cada entrada en `entrezGeneId` y `ensemblGeneId`

Descartemos primero los nombres que refieren a zonas intergénicas

In [8]:
df_genes = df_genes[ # Nos quedamos las columnas tal que ...
    ~( # ... NO se cumple que ...
        (df_genes["geneName"] == "intergenic") # ... el nombre del gen sea "intergenic" ...
        | (df_genes["geneName"] == "Intergenic") # ... O el nombre del gen sea "Intergenic"
    )
]
df_genes

,geneName,entrezGeneId,ensemblGeneId
0,GLG1,[{'entrezGeneId': '2734'}],[{'ensemblGeneId': 'ENSG00000090863'}]
2,RAD51L1,[],[]
3,RAD51L1,[],[]
4,FGFR2,[{'entrezGeneId': '2263'}],[{'ensemblGeneId': 'ENSG00000066468'}]
5,RAD51L1,[],[]
...,...,...,...
1615,L3MBTL3,[{'entrezGeneId': '84456'}],[{'ensemblGeneId': 'ENSG00000198945'}]
1616,ADCY9,[{'entrezGeneId': '115'}],[{'ensemblGeneId': 'ENSG00000162104'}]
1617,BAIAP2,[{'entrezGeneId': '10458'}],[{'ensemblGeneId': 'ENSG00000175866'}]
1619,COX11,[{'entrezGeneId': '1353'}],[{'ensemblGeneId': 'ENSG00000166260'}]


Hemos vuelto a reducir aún más el conjunto de datos: a 1301 entradas. El siguiente paso es extraer los identificadores

In [9]:
df_genes["entrezGeneId"] = df_genes["entrezGeneId"].apply( # Aplicamos función a la columna entrezGeneId
    lambda val: val[0]["entrezGeneId"] # Extraemos de la fila el ID de entrez ...
        if val != [] # ... si obtenemos una lista vacía ...
        else np.nan # ... añadimos un "Not a Number"
)

# Realizamos lo mismo para los identificadores de ensembl
df_genes["ensemblGeneId"] = df_genes["ensemblGeneId"].apply(
    lambda val: val[0]["ensemblGeneId"]
        if val != []
        else np.nan
)
df_genes

/tmp/ipykernel_152/3393425745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genes["entrezGeneId"] = df_genes["entrezGeneId"].apply( # Aplicamos función a la columna entrezGeneId
/tmp/ipykernel_152/3393425745.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genes["ensemblGeneId"] = df_genes["ensemblGeneId"].apply(


,geneName,entrezGeneId,ensemblGeneId
0,GLG1,2734,ENSG00000090863
2,RAD51L1,NaN,NaN
3,RAD51L1,NaN,NaN
4,FGFR2,2263,ENSG00000066468
5,RAD51L1,NaN,NaN
...,...,...,...
1615,L3MBTL3,84456,ENSG00000198945
1616,ADCY9,115,ENSG00000162104
1617,BAIAP2,10458,ENSG00000175866
1619,COX11,1353,ENSG00000166260


Además hemos detectado que hay filas idénticas. Vamos a eliminarlas.

In [11]:
df_genes.drop_duplicates(inplace=True)
df_genes

/tmp/ipykernel_152/1453503608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genes.drop_duplicates(inplace=True)


,geneName,entrezGeneId,ensemblGeneId
0,GLG1,2734,ENSG00000090863
2,RAD51L1,NaN,NaN
4,FGFR2,2263,ENSG00000066468
7,TOX3,27324,ENSG00000103460
10,MAP3K1,4214,ENSG00000095015
...,...,...,...
1610,CCNE1,898,ENSG00000105173
1611,TSPAN16,26526,ENSG00000130167
1613,CDH2,1000,ENSG00000170558
1616,ADCY9,115,ENSG00000162104


Con esta operación hemos reducido drásticamente el número de entradas.

El siguiente paso sería intentar conseguir los identificadores de aquellos genes que hemos marcado con `NaN`, pero no vamos a hacerlo puesto que implicaría meternos llamadas a APIs de Entrez, lo cual se sale del propósito de este curso. Seleccionemos todos los `ensemblGeneId` conocidos para consultar los términos Gene Ontology a través de la base de datos Panther.

In [12]:
# Lista de IDs de ENSEMBL
listaEnsemblIds = df_genes.ensemblGeneId.dropna().tolist()

# Ontologías interesantes a consultar
ontologies = {
    "biological_process" : "GO:0008150",
    "molecular_function" : "GO:0003674",
    "cellular_component" : "GO:0005575"
}
# El identificador del taxón humano en la base de datos de Panther
humanTaxonId = "9606"

# Hacemos la llamada a la base de datos
pantherReq = requests.post(
    r"http://pantherdb.org/services/oai/pantherdb/enrich/overrep",
    data={
        "geneInputList" : ",".join(listaEnsemblIds),
        "organism" : humanTaxonId,
        "annotDataSet" : ontologies["biological_process"]
    }
)

Si la llamada ha sido exitosa, cargamos la respuesta a un diccionario:

In [13]:
pantherResults = json.loads(pantherReq.text)
pantherResults["results"]["result"]

[{'number_in_list': 95,
  'fold_enrichment': 2.0048204947387624,
  'fdr': 3.5934705206791697e-09,
  'expected': 47.38578852785468,
  'number_in_reference': 5029,
  'pValue': 2.2929240177891587e-13,
  'term': {'id': 'GO:0048856', 'label': 'anatomical structure development'},
  'plus_minus': '+'},
 {'number_in_list': 97,
  'fold_enrichment': 1.8511958280884735,
  'fdr': 1.3956463547714149e-07,
  'expected': 52.39856233911311,
  'number_in_reference': 5561,
  'pValue': 1.7810698759206416e-11,
  'term': {'id': 'GO:0032502', 'label': 'developmental process'},
  'plus_minus': '+'},
 {'number_in_list': 98,
  'fold_enrichment': 1.8281998358198948,
  'fdr': 1.3351400367079596e-07,
  'expected': 53.60464325610763,
  'number_in_reference': 5689,
  'pValue': 2.5557810809876717e-11,
  'term': {'id': 'GO:0051171',
   'label': 'regulation of nitrogen compound metabolic process'},
  'plus_minus': '+'},
 {'number_in_list': 67,
  'fold_enrichment': 2.263091667924429,
  'fdr': 1.5280653586336915e-07,
  '

Lo malo es que esta consulta no nos devuelve el resultado deseado: deseamos saber qué genes se encuentran en cada término GO. Curiosamente, si hacemos esta consulta a mano directamente en [el formulario de Gene Ontology](http://geneontology.org/) y nos descargamos el fichero resultante como `.json`, sí que se nos proporciona esta información. Entonces, de manera excepcional, procederemos manualmente: ejecutamos el siguiente código, copiamos y pegamos el resultado del código en el cuadro de búsqueda de la página principal de Gene Ontology, elegimos `biological process` y `Homo sapiens`, y finalmente le damos a `Launch`.

In [14]:
for ID in listaEnsemblIds:
    print(ID)

ENSG00000090863
ENSG00000066468
ENSG00000103460
ENSG00000095015
ENSG00000091831
ENSG00000136826
ENSG00000033867
ENSG00000130592
ENSG00000186895
ENSG00000108175
ENSG00000138311
ENSG00000134461
ENSG00000147889
ENSG00000093144
ENSG00000112996
ENSG00000178568
ENSG00000140718
ENSG00000139618
ENSG00000074527
ENSG00000164330
ENSG00000142655
ENSG00000087494
ENSG00000164362
ENSG00000133067
ENSG00000170382
ENSG00000162426
ENSG00000050730
ENSG00000181788
ENSG00000148737
ENSG00000198625
ENSG00000154080
ENSG00000136770
ENSG00000164749
ENSG00000113448
ENSG00000144354
ENSG00000180530
ENSG00000166260
ENSG00000186998
ENSG00000130511
ENSG00000166446
ENSG00000015133
ENSG00000198807
ENSG00000136997
ENSG00000050327
ENSG00000010017
ENSG00000164379
ENSG00000152932
ENSG00000168594
ENSG00000168769
ENSG00000163513
ENSG00000150995
ENSG00000231672
ENSG00000172878
ENSG00000188761
ENSG00000185630
ENSG00000134207
ENSG00000065320
ENSG00000113369
ENSG00000161551
ENSG00000145491
ENSG00000160117
ENSG00000089225
ENSG0000

En la página resultante (¡no la cerréis!), pulsamos el botón de exportar como `JSON with user input ids`. Eso nos descargará un fichero llamado `analysis.json`, el cual movemos a nuestra carpeta de trabajo y abrimos a continuación:

In [15]:
with open("analysis.json") as file:
    analysisRes = json.load(file)

analysisRes

{'overrepresentation': {'reference': '',
  'upload_lists': {'input_list': {'organism': 'Homo sapiens',
    'list_name': 'upload_1'}},
  'tool_release_date': 20220202,
  'data_version_release_date': 'GO Ontology database DOI:  10.5281/zenodo.6399963 Released 2022-03-22',
  'annotation_type': 'GO biological process complete',
  'correction': 'FDR',
  'test_type': 'FISHER',
  'group': [{'result': [{'input_list': {'number_in_list': 2,
       'fold_enrichment': 106.12886597938144,
       'fdr': 0.03440793986327695,
       'expected': 0.018845014328039245,
       'pValue': 0.0005137479535481628,
       'mapped_id_list': {'mapped_id': ['ENSG00000066468', 'ENSG00000070193']},
       'list_name': 'upload_1',
       'plus_minus': '+'},
      'number_in_reference': 2,
      'term': {'level': 0,
       'id': 'GO:0060915',
       'label': 'mesenchymal cell differentiation involved in lung development'}},
     {'input_list': {'number_in_list': 70,
       'fold_enrichment': 1.9975855387353323,
      

Vamos a extraer todos los términos GO y los genes que pertenecen a los mismos:

In [16]:
# Copiamos el dataframe a un objeto nuevo.
df_genesGO = df_genes.copy(deep=True)

# Guardamos los grupos de análisis
analysisGroups = analysisRes["overrepresentation"]["group"]

# Diccionario para traducir los términos GO
goToWords = {}

# Función que nos ayudará a desempacar los datos a una matriz
def unpackGO(df, GOmapper, element):
    """Inputs:
    - df : el data frame donde vamos a guardar la información
    - GOmapper : el diccionario donde guardaremos los significados de GO
    """

    # Probar a ver si existe el ID
    try:
        GOTerm = element["term"]["id"]
    
    # Si falla, pues se usa el nombre común
    except:
        GOTerm = element["term"]["label"]

    # Obtener los genes que están anotados por dicho GO
    genesInGO = element["input_list"]["mapped_id_list"]["mapped_id"]

    # Guardar el significado del término GO
    GOmapper[GOTerm] = element["term"]["label"]

    # Crear columna de 0s
    df[GOTerm] = 0
    # Poner un 1 en cada gen que está anotado por dicho GO
    df.loc[
        df["ensemblGeneId"].isin(genesInGO),
        GOTerm
    ] = 1
    
    return (df, GOmapper)



# Para cada elemento en los grupos
for el in analysisGroups:
    
    # Si se trata de una lista
    if type(el["result"]) is list:
        # Itera sobre cada elemento de la lista
        for el2 in el["result"]:
            # Aplicando la función
            (df_genesGO, goToWords) = unpackGO(df_genesGO, goToWords, el2)

    # Si se trata de un dicconario
    if type(el["result"]) is dict:
        # Directamente aplica la función
        (df_genesGO, goToWords) = unpackGO(df_genesGO, goToWords, el["result"])

df_genesGO

/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ip

/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ip

/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ip

/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ip

/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ip

/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ipykernel_152/4125117372.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GOTerm] = 0
/tmp/ip

,geneName,entrezGeneId,ensemblGeneId,GO:0060915,GO:0048731,GO:0048856,GO:0032502,GO:0007275,GO:0032501,GO:0035295,...,GO:0048584,GO:0030182,GO:0048699,GO:0022008,GO:0009968,GO:0010648,GO:0023057,GO:0048585,GO:0042981,UNCLASSIFIED
0,GLG1,2734,ENSG00000090863,0,1,1,1,1,1,0,...,0,0,0,0,1,1,1,1,0,0
2,RAD51L1,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,FGFR2,2263,ENSG00000066468,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,0,1,0
7,TOX3,27324,ENSG00000103460,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10,MAP3K1,4214,ENSG00000095015,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610,CCNE1,898,ENSG00000105173,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1611,TSPAN16,26526,ENSG00000130167,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1613,CDH2,1000,ENSG00000170558,0,1,1,1,1,1,1,...,1,0,0,1,1,1,1,1,0,0
1616,ADCY9,115,ENSG00000162104,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


Como paso adicional, todos aquellos genes que tienen `NaN`, los incluiremos en `UNCLASSIFIED`

In [17]:
df_genesGO.loc[
    df_genesGO["ensemblGeneId"].isna(),
    "UNCLASSIFIED"
] = 1

df_genesGO

,geneName,entrezGeneId,ensemblGeneId,GO:0060915,GO:0048731,GO:0048856,GO:0032502,GO:0007275,GO:0032501,GO:0035295,...,GO:0048584,GO:0030182,GO:0048699,GO:0022008,GO:0009968,GO:0010648,GO:0023057,GO:0048585,GO:0042981,UNCLASSIFIED
0,GLG1,2734,ENSG00000090863,0,1,1,1,1,1,0,...,0,0,0,0,1,1,1,1,0,0
2,RAD51L1,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,FGFR2,2263,ENSG00000066468,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,0,1,0
7,TOX3,27324,ENSG00000103460,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
10,MAP3K1,4214,ENSG00000095015,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610,CCNE1,898,ENSG00000105173,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1611,TSPAN16,26526,ENSG00000130167,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1613,CDH2,1000,ENSG00000170558,0,1,1,1,1,1,1,...,1,0,0,1,1,1,1,1,0,0
1616,ADCY9,115,ENSG00000162104,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


Además tenemos el texto explicativo de cada Gene Ontology

In [18]:
goToWords

{'GO:0060915': 'mesenchymal cell differentiation involved in lung development',
 'GO:0048731': 'system development',
 'GO:0048856': 'anatomical structure development',
 'GO:0032502': 'developmental process',
 'GO:0007275': 'multicellular organism development',
 'GO:0032501': 'multicellular organismal process',
 'GO:0035295': 'tube development',
 'GO:0048513': 'animal organ development',
 'GO:0030154': 'cell differentiation',
 'GO:0048869': 'cellular developmental process',
 'GO:0009987': 'cellular process',
 'GO:0060485': 'mesenchyme development',
 'GO:0009888': 'tissue development',
 'GO:0060667': 'branch elongation involved in salivary gland morphogenesis',
 'GO:0022612': 'gland morphogenesis',
 'GO:0048732': 'gland development',
 'GO:0009887': 'animal organ morphogenesis',
 'GO:0009653': 'anatomical structure morphogenesis',
 'GO:0061138': 'morphogenesis of a branching epithelium',
 'GO:0001763': 'morphogenesis of a branching structure',
 'GO:0002009': 'morphogenesis of an epitheliu

A continuación, con la información de la presencia y ausencia de un gen en un Gene Ontology, construimos una matriz de adyacencia con pesos:

In [19]:
# Obtenemos el nombre de los nodos a partir de la diferencia de columnas
nodes = sorted(
    list(
        set(df_genesGO.columns) - set(df_genes.columns)
    )
)

# Creamos matriz de adyacencia vacía 
adjmat = np.zeros((len(nodes), len(nodes)))

# Por cada nodo...
for i, el1 in enumerate(nodes):
    # Y por cada otro nodo ...
    for j, el2 in enumerate(nodes[i+1:], start=i+1):
        
        # añadimos el número de genes que comparten ese GO
        adjmat[i, j] = sum(df_genesGO[el1] & df_genesGO[el2])

# Le sumamos la traspuesta puesto que es una matriz triangular superior
# y el grafo es no dirigido
adjmat = adjmat + adjmat.T

Con la matriz de adyacencia podemos crear nuestro grafo

In [20]:
# crear grafo
G = nx.from_numpy_matrix(adjmat)
# Poner nombre adecuado a los nodos
G = nx.relabel_nodes(
    G,
    dict(enumerate(nodes))
)

El siguiente paso consiste en dibujar el grafo. Esta parte la podéis hacer vosotros mirando el ejemplo del fichero `Drug-Drug BioSNAP.ipynb` por ejemplo